# Total Graduates in Education

## Map and Graph

Will use Dash recreate the graph found H:\CALDER\CALDER Data Visualizations\Jupyter Notebooks\Teacher Labor Shortage\IPEDS with file name: Map - (Awlevel State) Aggregated Number of Degrees in Education (1984-2016). 

There was a problem rendering the file and I haven't found out why. 

In [1]:
# Dependencies for project
import dash
import dash_core_components as dcc
import dash_html_components as html
import plotly.graph_objs as go
import pandas as pd
from dash.dependencies import Input, Output, State

### Import data

Reshaped data to long using Stata (H:\CALDER\CALDER Data Visualizations\Data\Teacher Labor Shortage) using file *Reshape_Aggregated Number of Degrees in Education.do*

In [2]:
import os

In [3]:
os.chdir(r"H:\CALDER\CALDER Data Visualizations\Data\Teacher Labor Shortage")
excel = pd.read_excel('Aggregated Number of Degrees in Education (Reshaped long long).xlsx')

In [4]:
excel.head(5)

,year,state_short,state_long,indicator,value
0,1984,AK,Alaska,elem ba,84
1,1984,AK,Alaska,other ba,24
2,1984,AK,Alaska,sped ba,0
3,1984,AK,Alaska,stem ba,0
4,1984,AK,Alaska,elem ma,5


In [5]:
excel.indicator.unique()

array(['elem ba', 'other ba', 'sped ba', 'stem ba', 'elem ma', 'other ma',
       'sped ma', 'stem ma', 'elem phd', 'other phd', 'sped phd',
       'stem phd', 'agg total', 'ba total', 'elem total', 'ma total',
       'other total', 'phd total', 'sped total', 'state total',
       'stem total'], dtype=object)

In [6]:
excel.replace('elem ba', 'Elementary Bachelor', inplace=True)
excel.replace('other ba', 'Other Bachelor', inplace=True)
excel.replace('sped ba', 'SPED Bachelor', inplace=True)
excel.replace('stem ba', 'STEM Bachelor', inplace=True)
excel.replace('elem ma', 'Elementary Masters', inplace=True)
excel.replace('other ma', 'Other Masters', inplace=True)
excel.replace('sped ma', 'SPED Masters', inplace=True)
excel.replace('stem ma', 'STEM Masters', inplace=True)
excel.replace('elem phd', 'Elementary PhD', inplace=True)
excel.replace('other phd', 'Other PhD', inplace=True)
excel.replace('sped phd', 'SPED PhD', inplace=True)
excel.replace('stem phd', 'STEM PhD', inplace=True)
excel.replace('agg total' , 'National Total', inplace=True)

excel.replace('ba total', 'Bachelor Total', inplace=True)
excel.replace('elem total', 'Elementary Total', inplace=True)
excel.replace('ma total', 'Masters Total', inplace=True)
excel.replace('other total', 'Other Total', inplace=True)
excel.replace('phd total', 'PhD Total', inplace=True)
excel.replace('sped total', 'SPED Total', inplace=True)
excel.replace('state total', 'State Total', inplace=True)
excel.replace('stem total', 'STEM Total', inplace=True)

In [7]:
excel2 = excel[(excel['indicator'] == 'Bachelor Total') | (excel['indicator']=='Masters Total') 
               | (excel['indicator']=='PhD Total')   | (excel['indicator']=='Elementary Total')
               | (excel['indicator']=='Other Total') | (excel['indicator']=='SPED Total')
               | (excel['indicator']=='State Total') | (excel['indicator']=='STEM Total')]

### Export
Export data so it can be used elsewhere

In [ ]:
excel2.to_excel('')

## Create Graph

Once I've tested the code I will transfer this to a .py file and follow the steps found here: https://plot.ly/dash/deployment in order to get the figure on the [CALDER Website](https://caldercenter.org/data-visualizations)

### Version 1

This contains:
* Two dropdowns, one for state and one for all indicators selected above
* One graph
* You can select more than one indicator but only one state

In [8]:
app = dash.Dash()

app.css.append_css({"external_url": "https://codepen.io/chriddyp/pen/bWLwgP.css"}) 

df = excel2

state = df['state_long'].unique()
available_indicators = ['State Total', 'Bachelor Total', 'Masters Total', 'PhD Total', 'Elementary Total', 'SPED Total', 
                        'STEM Total', 'Other Total']

# Organize where items will be on the page
app.layout = html.Div([
        html.H3(
            children='Aggregated Number of Graduates in Education by State',
            style={
                'textAlign': 'center', 'fontFamily' : 'Georgia'
            }
        ),
    
        
        html.Div([          
            html.Div([
                html.Div([html.P('Select a State from the drop down below',id='state-title')],
                    style={'textAlign': 'center', 'fontFamily': 'Georgia'}),
                dcc.Dropdown(
                    id='state-id',
                    options=[{'label': i, 'value': i} for i in state],
                    value = 'Alaska'
                )
            ],
            style={'width': '48%', 'display': 'inline-block'}),
            
        html.Div([
            html.Div([html.P('Select Totals by broad CIP categories and Award type',id='indicator-title')],
                style={'textAlign': 'center', 'fontFamily' : 'Georgia'}),
            dcc.Dropdown(
                id='indicator-id',
                options=[{'label': i , 'value': i} for i in available_indicators],
                value=['State Total'],
                multi=True # This treats items as a list. 
                )
            ],
            style={'width':'48%', 'float': 'right', 'display': 'inline-block'})
        ]),
        
        dcc.Graph(id='indicator-graphic')
    ])


@app.callback(

    dash.dependencies.Output('indicator-graphic', 'figure'),
    [dash.dependencies.Input('state-id', 'value'),
     dash.dependencies.Input('indicator-id', 'value')])
def update_time_series(state_id, indicator_ids):
    dff = df[df['state_long'] == state_id]
    lines = {}
    data = []
    for indicator_id in indicator_ids:
        if indicator_id=='Bachelor Total':  
            lines = dict(
                 color = ("#6b6ecf"),
                 width = 3,
                 dash = 'dash')
        if indicator_id=='Masters Total':           
            lines = dict(
                 color = ("#80b1d3"),
                 width = 3,
                 dash = 'dash')
        if indicator_id=='State Total':           
                lines = dict(
                     color = ("#333333"),
                     width = 3)
        if indicator_id=='PhD Total':           
                lines = dict(
                     color = ("#35B778"),
                     width = 3,
                     dash = 'dash') 
        if indicator_id=='SPED Total':           
                lines = dict(
                     color = ("#fdb462"),
                     width = 3,
                     dash = 'dot')
        if indicator_id=='Elementary Total':           
                lines = dict(
                     color = ("#bebada"),
                     width = 3,
                     dash = 'dot')
        if indicator_id=='Other Total':           
                lines = dict(
                     color = ("#fb8072"),
                     width = 3,
                     dash = 'dot')
        if indicator_id=='STEM Total':           
                lines = dict(
                     color = ("#8dd3c7"),
                     width = 3,
                     dash = 'dot') 
        trace = go.Scatter(
            x = dff[dff['indicator'] == indicator_id]['year'],
            y = dff[dff['indicator'] == indicator_id]['value'],
            mode='lines',
            name = indicator_id,
            line = lines,
            opacity = 0.8
            )
        
        data.append(trace)
    return {
        'data' : data,
        'layout' : go.Layout(
            xaxis={'title': 'Year'},
            yaxis={'title': 'Total Graduates in Education'}
        )
    }

if __name__ == '__main__':
    app.run_server()

 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [15/Feb/2018 12:47:46] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [15/Feb/2018 12:47:47] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [15/Feb/2018 12:47:47] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [15/Feb/2018 12:47:47] "GET /favicon.ico HTTP/1.1" 200 -
127.0.0.1 - - [15/Feb/2018 12:47:47] "POST /_dash-update-component HTTP/1.1" 200 -


## Test

Make changes and merge them to the docket above. 